# 3. Multi-layer perceptron

- Used as part of INFO8010 Deep Learning (Gilles Louppe, 2018-2019).
- Originally adapted from [Pytorch tutorial for Deep Learning researchers](https://github.com/yunjey/pytorch-tutorial) (Yunvey Choi, 2018).

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Hyper-parameters

In [ ]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# Data

In [ ]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Model

In [ ]:
# Model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = Net(input_size, hidden_size, num_classes)

# Loss and optimizer

In [ ]:
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                  % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss))

# Tensorboard

As you can see, evaluating the learning of your model ask you to take a look at the loss evolution during training time. [Tensorboard](https://www.tensorflow.org/guide/summaries_and_tensorboard), more specifically [tensorboardX](https://github.com/lanpa/tensorboardX) on pytorch, is a toolkit which is made to simplify this procedure. 

In [ ]:
from tensorboardX import SummaryWriter
import time

In [ ]:
# Initialize the data logger
writer = SummaryWriter() 

In [ ]:
for i in range(50):
    writer.add_scalar('data/scalar1', i**2, i)
    time.sleep(.5)

<div class="alert alert-success">
<b>EXERCISE</b>:

Now that you have seen how this handy tool works, you can monitor the progress of your neural networks within it. 
Redirect what you think the most informative logs of your training are to tensorboard by creating an appropriate writer.

</div>

# Test the model

In [ ]:
# Test the Model
correct = 0
total = 0

for images, labels in test_loader:
    images = images.view(-1, 28*28)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

<div class="alert alert-success">
<b>EXERCISE</b>:

Compare the performance of an MLP with sigmoid activation units against a rectified network.

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:
    
What is the purpose of using non linear activations?
</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

What is the effect of the width of the hidden layers?

</div>

<div class="alert alert-success">
<b>EXERCISE</b>:

What is the effect of the depth of network?

</div>

## Vanishing gradient (bonus)

<div class="alert alert-success">
<b>EXERCISE</b>:

Very deep networks are known to be affected by the "Vanishing Gradient" problem.
Investigate this phenomenon by defining an architecture which would suffer from this issue and compare the gradients that you obtain with the ones of a shallower network by using the weight.grad.norm() method.
Plot the results.


</div>

In [ ]:
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        print(model.fc1.weight.grad.norm())
        #break
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                  % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss))